# MySQL Database Query Interface

This notebook provides a mini-interface to a MySQL database named `ich_edit` containing three tables: Users, Products, and Sales. The interface allows querying single tables or joining

In [ ]:
import os
import pym

## Database Structure
- **Users**: id, name, age
- **Products**: pid, prod, quantity
- **Sales**: sid, id, pid

The mini-interface allows:
1. Selecting multiple tables and joining them if possible
2.

In [ ]:
import mysql.connector
from dotenv import load_dotenv
import os

# Install required packages if not already installed
try:
    import mysql.connector
except ImportError:
    !pip install mysql-connector-python
    import mysql.connector

try:
    from dotenv import load_dotenv
except ImportError:
    !pip install python-dotenv
    from dotenv import load_dotenv

# Load environment variables
load_dotenv('../.env')

# Database configuration
dbconfig = {
    'host': os.getenv('ICH_EDIT_HOST'),
    'user': os.getenv('ICH_EDIT_USER'),
    'password': os.getenv('ICH_EDIT_PASSWORD'),
    'database': 'ich_edit',
}

In [ ]:
def prepare_query(tables):
    tables = [t.strip() for t in tables.split(',')]

    if len(tables) > 1:
        # Define join relationships
        join_map = {
            ('Users', 'Sales'): 'Users.id = Sales.id',
            ('Sales', 'Users'): 'Users.id = Sales.id',
            ('Sales', 'Products'): 'Sales.pid = Products.pid',
            ('Products', 'Sales'): 'Sales.pid = Products.pid',
            # Handle additional possible combinations
            ('Users', 'Products'): None,
            ('Products', 'Users'): None
        }

        # Check if all pairs of tables can be joined
        can_be_joined = True
        for i in range(len(tables) - 1):
            pair = (tables[i], tables[i+1])
            if pair not in join_map or join_map[pair] is None:
                can_be_joined = False
                break

        if not can_be_joined:
            return "ERROR: Tables cannot be joined directly"

        # Build the query with proper joins
        query = f"SELECT * FROM {tables[0]}"
        for i in range(1, len(tables)):
            pair = (tables[i-1], tables[i])
            join_condition = join_map[pair]
            query += f" JOIN {tables[i]} ON {join_condition}"

        return query
    else:
        # Single table query with optional field search
        table = tables[0]
        print(f"\nAvailable fields in {table}:")
        field_maps = {
            'Users': ['id', 'name', 'age'],
            'Products': ['pid', 'prod', 'quantity'],
            'Sales': ['sid', 'id', 'pid']
        }
        print(field_maps.get(table, []))

        search = input("\nDo you want to search by field value? (y/n): ")
        if search.lower() == 'y':
            field = input("Enter field name: ")
            value = input("Enter search value: ")

            # Determine if value should be quoted (string) or not (number)
            try:
                float(value)  # Check if value is a number
                return f"SELECT * FROM {table} WHERE {field} = {value}"
            except ValueError:
                return f"SELECT * FROM {table} WHERE {field} = '{value}'"

        return f"SELECT * FROM {table}"

In [ ]:
# Main interface
print("\nAvailable tables: Users, Products, Sales")
input_tables = input("Enter tables (comma-separated): ")

prepared_query = prepare_query(input_tables)
print(f"\nExecuting query: {prepared_query}")

if not prepared_query.startswith("ERROR"):
    try:
        with mysql.connector.connect(**dbconfig) as connection:
            with connection.cursor() as cursor:
                cursor.execute(prepared_query)
                result = cursor.fetchall()

                # Get column names for better display
                column_names = [desc[0] for desc in cursor.description]
                print("\nColumns:", column_names)

                print("\nResults:")
                if result:
                    for row in result:
                        print(row)
                else:
                    print("No results found.")
    except mysql.connector.Error as err:
        print(f"Database error: {err}")
else:
    print(prepared_query

## Example Usage:

1. To query a single table:
   - Enter: `Users`
   - Choose whether to filter by field value

2. To join tables:
   - Enter: `Users, Sales` or `Sales, Products`
   - These will join automatically using the defined relationships

3. Invalid joins:
   - Enter: `Users, Products`
   - You'll get an error as these tables don't have a direct relationship